# Assignment 1
- s113604 Mark Ian Svenningsen
- s113627 Casper Clemmensen
- s146893 Kasper Hoa Quoc Duong

## Assignment 1A: Anscombe's quartet

## Assignment 1B: Slicing data

### We'll be combining information about PdDistrict and Category to explore differences between SF's neighborhoods. First, simply list the names of SF's 10 police districts.

We begin by loading the cvs file. Containing all the crime records, and parse it into a dictonary, for easy access.

Note that we ignore any crime that do not have a district.

In [1]:
from csv import reader

def parseCrimeCsv(row):
    return {
        'IncidntNum': line[0],
        'Category': line[1],
        'Descript': line[2],
        'DayOfWeek': line[3],
        'Date': line[4],
        'Time': line[5],
        'PdDistrict': line[6],
        'Resolution': line[7],
        'Address': line[8],
        'X': line[9],
        'Y': line[10],
        'Location': line[11],
        'PdId': line[12]
    }

with open('../SFPD_Incidents_-_from_1_January_2003.csv', 'r') as infile:    
    data = [parseCrimeCsv(line) for line in list(reader(infile))[1:] if not line[6] == '']

Now that the file have been loaded into memory we can extract all the districts.
We do this by taking value ` PdDistrict ` of all records, and insert them into a set.

In [2]:
PdDistrictSet = set([i['PdDistrict'] for i in data])
PdDistrictSet

{'BAYVIEW',
 'CENTRAL',
 'INGLESIDE',
 'MISSION',
 'NORTHERN',
 'PARK',
 'RICHMOND',
 'SOUTHERN',
 'TARAVAL',
 'TENDERLOIN'}

### Which has the most crimes? Which has the most focus crimes?

If we being finding the district with the most overall crimes.
We begin with creating a `Counter` object, were we input the districts of the dataset.
This object, helps us counting the occorences of the given values.

Lastly we call the ` most_common(1) ` on the `Counter` to extract the district with the most crimes overall.

In [7]:
from collections import Counter

DistrictCrimeCounter = Counter([d['PdDistrict'] for d in data])

DistrictCrimeCounter.most_common(1)

[('SOUTHERN', 335978)]

By looking at hthe output above, we can se that **Southern** is the district with the most crimes overall with a total count of 335978 crimes from Jan 2003 to Jan 2016.

** Which has the most focus crimes?**
To do this we first define the focus crimes, these are taken from the exercises from [week 3](http://nbviewer.jupyter.org/github/suneman/socialdataanalysis2016/blob/master/lectures/Week3.ipynb):

In [4]:
focuscrimes = set([
        'WEAPON LAWS',
        'PROSTITUTION',
        'DRIVING UNDER THE INFLUENCE',
        'ROBBERY',
        'BURGLARY',
        'ASSAULT',
        'DRUNKENNESS',
        'DRUG/NARCOTIC',
        'TRESPASS',
        'LARCENY/THEFT',
        'VANDALISM',
        'VEHICLE THEFT',
        'STOLEN PROPERTY',
        'DISORDERLY CONDUCT'
    ])

Now that we have defined the focus crimes, we can filter our dataset. So that we get a new data set that only contains the records for these crimes. We do this by utilizing the build in filter function.

Then we do the same operatios a above, to get the district with the most focus crimes.

In [8]:
data_filtered_for_focus_crimes = filter(lambda c: c['Category'] in list(focuscrimes), data)

       
most_common_focus_crime_by_dist = Counter([d['PdDistrict'] for d in data_filtered_for_focus_crimes])
most_common_focus_crime_by_dist.most_common(1)

[('SOUTHERN', 191521)]

We see that it is again the district south that have the most focus crimes, just as it also had with all crimes.
However this time around we are down to 191521 crimes commited between Jan 2003 and Jan 2016.

### Next, we want to generate a slightly more complicated graphic. I'm interested to know if there are certain crimes that happen much more in certain neighborhoods than what's typical. Below I describe how to get that plot going

#### First, we need to calculate the relative probabilities of seeing each type of crime in the dataset as a whole. That's simply a normalized version of this plot. Let's call it P(crime).

Firstly we have to create a `Counter` for the categories. Then we can use list comprehention, over the output of `most common`, to create a nicely structred dataset. 

In [13]:
from __future__ import division

# DistrictCrimeCounter

CatetoryCrimeCounter = Counter([d['Category'] for d in data])

p_crime = [{k: v/len(data)} for k,v in CatetoryCrimeCounter.most_common()]
p_crime

[{'LARCENY/THEFT': 0.20380476864336636},
 {'OTHER OFFENSES': 0.1426449149600833},
 {'NON-CRIMINAL': 0.10625637465623582},
 {'ASSAULT': 0.0875342429177903},
 {'VEHICLE THEFT': 0.060621045043120714},
 {'DRUG/NARCOTIC': 0.05924919232104237},
 {'VANDALISM': 0.05101914399380557},
 {'WARRANTS': 0.047586575174218354},
 {'BURGLARY': 0.04178837476303634},
 {'SUSPICIOUS OCC': 0.03587482978666595},
 {'MISSING PERSON': 0.029413932128267428},
 {'ROBBERY': 0.02584999866499346},
 {'FRAUD': 0.019067631431394012},
 {'FORGERY/COUNTERFEITING': 0.011604944864229834},
 {'SECONDARY CODES': 0.011447948094945665},
 {'WEAPON LAWS': 0.009805890048861239},
 {'TRESPASS': 0.008350732918591302},
 {'PROSTITUTION': 0.00831068272234534},
 {'STOLEN PROPERTY': 0.005342162176594665},
 {'SEX OFFENSES, FORCIBLE': 0.005046324726991162},
 {'DRUNKENNESS': 0.004795343497183136},
 {'DISORDERLY CONDUCT': 0.004770779376818946},
 {'RECOVERED VEHICLE': 0.0033887806050249647},
 {'DRIVING UNDER THE INFLUENCE': 0.002626224868501856},


#### Next, we calculate that same probability distribution but for each PD district, let's call that P(crime|district).

Now we do the same, however we do it per district instead.

Our result is a dictonary, with the district as key, followed by a list, containing dictionaries again, with the crime category as key and properbility as value.

In [16]:
p_crime_district = {}
for district in list(PdDistrictSet):
    counter = Counter([d['Category'] for d in data if d['PdDistrict'] == district])
    p_crime_district[district] = [{k: v/len(data)} for k,v in counter.most_common()]
    
    
p_crime_district

{'BAYVIEW': [{'OTHER OFFENSES': 0.019140255787253358},
  {'LARCENY/THEFT': 0.011709609377085949},
  {'ASSAULT': 0.011098176381064266},
  {'VEHICLE THEFT': 0.007864256534857022},
  {'NON-CRIMINAL': 0.0070995647878674606},
  {'VANDALISM': 0.006218994473072918},
  {'MISSING PERSON': 0.00553226710810883},
  {'DRUG/NARCOTIC': 0.004932582169652631},
  {'WARRANTS': 0.004928844151336342},
  {'BURGLARY': 0.004489359997863989},
  {'SUSPICIOUS OCC': 0.004435959736202707},
  {'ROBBERY': 0.0030128427629295385},
  {'WEAPON LAWS': 0.0019320214669051878},
  {'SECONDARY CODES': 0.001907991349157611},
  {'FRAUD': 0.001022615010813553},
  {'FORGERY/COUNTERFEITING': 0.0008597442127466424},
  {'RECOVERED VEHICLE': 0.000759351720823432},
  {'TRESPASS': 0.0007262435585934371},
  {'SEX OFFENSES, FORCIBLE': 0.00047312631831895977},
  {'ARSON': 0.00044963020318799563},
  {'STOLEN PROPERTY': 0.0004469601901049315},
  {'KIDNAPPING': 0.00037486983686220064},
  {'DRUNKENNESS': 0.00027768136063866715},
  {'RUNAWAY':

#### Now we look at the ratio P(crime|district)/P(crime). That ratio is equal to 1 if the crime occurs at the same level within a district as in the city as a whole. If it's greater than one, it means that the crime occurs more frequently within that district. If it's smaller than one, it means that the crime is rarer within the district in question than in the city as a whole.

#### For each district plot these ratios for the 14 focus crimes. My plot looks like this


#### Comment on the top crimes in Tenderloin, Mission, and Richmond. Does this fit with the impression you get of these neighborhoods on Wikipedia?

#### Even though we only plotted the ratios for our 14 focus crimes, I asked you to calculate the ratios based on all crime categories. Why do you think I wanted to include all crime types in the calculation?

## Assignment 1C: KNN

## Assignment 1D: Multiple regression and the Red Baron